# Global Functions

In [24]:
def register_packets():
    # needs memoization for global packet naming , a cache that sees if the name has been named before or not 
    Packets = []
    packet_info = ""
    previous_packet_names = []
    i = 0
    while True:
        i += 1
        packet_info = input(f"> Entry {i} : ")
        if packet_info == "-1":
            del i 
            break
        else:
            packet_info = packet_info.split(" | ")
            try:
                if packet_info[0] not in previous_packet_names:
                    previous_packet_names.append(packet_info[0])
                else:
                    raise ValueError("Duplicate Packet Entered ...")
                name = packet_info[0]
                arrival_time = int(packet_info[1])
                length = int(packet_info[2])
                Packets.append(Packet(name,arrival_time,length))
            except ValueError as e:
                print("Cannot Accept Numbers for Packet Name")
                del i 
                return None
            except IndexError as idxe:
                print("Missing Argument")
                del i 
                return None
    return Packets

## Packet Class

In [25]:
class Packet:
    def __init__(self,name,arrival_time,length):
        self.name = name
        self.arrival_time = arrival_time
        self.length = length
        self.weighted_length = 0 
        self.virtual_finish_time = float('inf')
        self.actual_finish_time = float('inf')
        self.order = -1

    def set_weighted_length(self,weight):
        assert self.length % weight == 0 , "Weighted length cannot be a float number"
        self.weighted_length =  self.length // weight
        
    def set_virtual_finish_time(self,previous_finish_time):
        self.virtual_finish_time = previous_finish_time + self.weighted_length

    def set_actual_finish_time(self,previous_finish_time):
        self.actual_finish_time = previous_finish_time + self.length

    def set_order(self,order):
        self.order = order 

    def print_packet(self):
        print(f" {self.name} | {self.arrival_time} | {self.length} | {self.weighted_length} | {self.virtual_finish_time} | {self.actual_finish_time} | {self.order}")

    def __str__(self):
        return f"({self.name} , {self.arrival_time} , {self.length} )"

## PacketQueue Class

In [26]:
class PacketQueue:
    id = 0

    @classmethod
    def reset_id(cls):
        cls.id = 0
    
    @classmethod
    def rename_queue(cls):
        cls.id += 1
        return "Q" + str(cls.id)
        
    def __init__(self,weight,packets):
        self.name = PacketQueue.rename_queue()
        self.weight = weight
        if packets is not None:
            self.packets = sorted(packets , key = lambda x : (x.arrival_time , ord(x.name)))
            for packet in self.packets:
                packet.set_weighted_length(weight)
        else:
            self.packets = None


    def calculate_virtual_finish_time(self):
        self.virtual_finish_times = []
        global_finish_time = 0
        for packet in self.packets:
            if packet.arrival_time >= global_finish_time : 
                packet.set_virtual_finish_time(packet.arrival_time)
            else:
                packet.set_virtual_finish_time(global_finish_time)
            global_finish_time = packet.virtual_finish_time
            self.virtual_finish_times.append((global_finish_time,PacketQueue.id,packet.name))
        return self.virtual_finish_times

    def get_packet(self,packet_name):
        match = filter(lambda p : p.name == packet_name , self.packets)
        match = list(match)
        return match[0]
    
    def print_queue(self):
        for packet in self.packets:
            packet.print_packet()

    def __str__(self):
        return f"( {self.name} , {self.weight} , {self.packets} )"

## WFQ class

In [29]:
class WFQ:
    def __init__(self,num_of_queues):
        PacketQueue.reset_id()
        print("Welcome to WFQ registry ...")
        print("Please enter required Packet to the format given below")
        print("Format: Packet_Name | Arrival Time | Length (-1 to exit)")
        self.pqueues = []
        self.queues_vlist = []
        for i in range(num_of_queues):
            print(f"Registry of Queue {i+1} packets")
            q_packets = register_packets()
            print(f"Please assign Queue{i+1} 's weight")
            weight = int(input(">"))
            q = PacketQueue(weight,q_packets)
            self.queues_vlist.extend(q.calculate_virtual_finish_time())
            self.pqueues.append(q)
        del q 
        del q_packets

    
    def assign_order(self):
        self.order_list = []
        self.queues_vlist = sorted(self.queues_vlist , key = lambda x : (x[0],x[1]))
        counter = 1
        for i in (self.queues_vlist):
            packet = self.pqueues[i[1]-1].get_packet(i[2])
            packet.set_order(counter)
            counter += 1
            self.order_list.append(packet)
        self.order_list.reverse()
        del self.queues_vlist
                

    def calculate_actual_finish_time(self):
        global_finish_time = 0
        while self.order_list:
            # because pop starts from last element which is technically the first packet now
            p = self.order_list.pop()
            if p.arrival_time >= global_finish_time:
                p.set_actual_finish_time(p.arrival_time)
            else:
                p.set_actual_finish_time(global_finish_time)
            global_finish_time = p.actual_finish_time
        del self.order_list
                
    
    def print_WFQ(self):
        print(" N |Arv| L |W.L| VFT | AFT | order")
        for packet_queue in self.pqueues:
            print(f"Queue: {packet_queue.name} has the following packets")
            packet_queue.print_queue()
        
            
    

In [28]:
wfq = WFQ(3)
wfq.assign_order()
wfq.calculate_actual_finish_time()
wfq.print_WFQ()

Welcome to WFQ registry ...
Please enter required Packet to the format given below
Format: Packet_Name | Arrival Time | Length (-1 to exit)
Registry of Queue 1 packets


> Entry 1 :  A | 0 | 2
> Entry 2 :  D | 3 | 4
> Entry 3 :  G | 6 | 10
> Entry 4 :  J | 6 | 6
> Entry 5 :  -1


Please assign Queue1 's weight


> 2


Registry of Queue 2 packets


...

Registry of Queue 3 packets


> Entry 1 :  C | 0 | 18
> Entry 2 :  F | 3 | 6
> Entry 3 :  I | 5 | 3
> Entry 4 :  K | 7 | 9
> Entry 5 :  -1


Please assign Queue3 's weight


> 3


 N |Arv| L |W.L| VFT | AFT | order
Queue: Q1 has the following packets
 A | 0 | 2 | 1 | 1 | 2 | 1
 D | 3 | 4 | 2 | 5 | 7 | 2
 G | 6 | 10 | 5 | 11 | 49 | 7
 J | 6 | 6 | 3 | 14 | 67 | 10
Queue: Q2 has the following packets
 B | 4 | 5 | 5 | 9 | 36 | 5
 E | 4 | 3 | 3 | 12 | 52 | 8
 H | 7 | 2 | 2 | 14 | 69 | 11
Queue: Q3 has the following packets
 C | 0 | 18 | 6 | 6 | 25 | 3
 F | 3 | 6 | 2 | 8 | 31 | 4
 I | 5 | 3 | 1 | 9 | 39 | 6
 K | 7 | 9 | 3 | 12 | 61 | 9
